In [1]:
from scipy.stats import uniform
from scipy.stats import norm
from scipy.stats import truncnorm
import numpy as np

#DISCLAIMER: CODED FOR VAR OF ORDER 1

## Jumping distribution of theta, conditioned on all values except index j
def jump_dst(theta_old,j,user_std,K):
    #theta_old: previous value of vector theta
    #j: index for which dist is unconditioned
    #user_std: size of step of jumping distribution
    
    dt = 0.0001 #avoid exactly taking limits of bounds

    mu = theta_old[j]
    if (j < (K*K*2)):
        rv = norm(loc=mu,scale=user_std)
    elif ( (j >= (K*K*2)) and (j < (K*K*2+K)) ):
        a, b = (-1+dt - mu) / user_std, (1-dt - mu) / user_std
        rv = truncnorm(a=a,b=b,loc=mu,scale=user_std) #bounded between (-1,1)
    elif ( (j >= (K*K*2+K)) and (j < (K*K*2+K*2)) ):
        a  = (0+dt - mu) / user_std
        rv = truncnorm(a=a,b=np.inf,loc=mu,scale=user_std) #bounded between (0,+inf) 
    else:
        print("ERROR: index j out of bounds")

    theta = theta_old.copy()
    theta[j] = rv.rvs()

    samp_vecA = np.reshape(theta[:(K*K)],(K,K))
    samp_vecU = np.reshape(theta[(K*K):(K*K*2)],(K,K))
    samp_valA = np.diag(theta[(K*K*2):(K*K*2+K)])
    samp_valU = np.diag(theta[(K*K*2+K):(K*K*2+K*2)])

    A = samp_vecA @ samp_valA @np.linalg.inv(samp_vecA)
    U = samp_vecU @ samp_valU @np.linalg.inv(samp_vecU)
    
    return(A,U,theta)

In [2]:
## Previous values

#matriz A
A_old = np.array([[ 0.19342949,0.06497526,0.20640138],
 [ 0.71551897,-0.8875307,0.81804304],
 [ 0.55257784,-0.47167893,0.54647069]])
#matriz de vectores propios de A
samp_vecA_old = np.array([[-0.61605662,1.60194096,1.54104974],
 [ 3.0063165,1.95457467,-0.34768391],
 [ 1.49965918,1.98081968,-1.70410413]])
#matriz de valores propios de A
samp_valA_old = np.array([[-0.62608599,0.      ,0.        ],
 [ 0.       ,0.52792554,0.        ],
 [ 0.       ,0.     ,-0.04947007]])

#matriz U
U_old = np.array([[ 0.2033993,-0.18571928,0.33787476],
 [ 0.19604154,-1.15594823,2.57610726],
 [ 0.10936385,-0.74142332,1.61134019]])
#matriz de vectorios de propios U
samp_vecU_old = np.array([[ 0.04359329,1.03759474,-1.67779624],
 [ 0.97607584,0.64018012,1.39118497],
 [ 0.54931646,0.25065424,0.8522514 ]])
#matriz de valores propios U
samp_valU_old = np.array([[0.30259028,0.   ,0. ],
 [0.    ,0.17043456,0.  ],
 [0.    ,0.     ,0.18576643]])

#vector theta que contiene la info
theta_old = np.concatenate((samp_vecA_old.reshape(-1),samp_vecU_old.reshape(-1),
                            np.diag(samp_valA_old),np.diag(samp_valU_old)))

In [3]:
## User Parameters
K =  3 #problem dimension(number of wind farms)
j =  22 #index which is left unconditioned of vector theta, j = 1,...,K(K+1)
user_std = 1

out = jump_dst(theta_old,j,user_std,K)

#print(out[0])
#print(A_old)
#print(out[1])
#print(U_old)
print(theta_old)
print(out[2])

[-0.61605662  1.60194096  1.54104974  3.0063165   1.95457467 -0.34768391
  1.49965918  1.98081968 -1.70410413  0.04359329  1.03759474 -1.67779624
  0.97607584  0.64018012  1.39118497  0.54931646  0.25065424  0.8522514
 -0.62608599  0.52792554 -0.04947007  0.30259028  0.17043456  0.18576643]
[-0.61605662  1.60194096  1.54104974  3.0063165   1.95457467 -0.34768391
  1.49965918  1.98081968 -1.70410413  0.04359329  1.03759474 -1.67779624
  0.97607584  0.64018012  1.39118497  0.54931646  0.25065424  0.8522514
 -0.62608599  0.52792554 -0.04947007  0.30259028  0.74761964  0.18576643]
